In [11]:
import tifffile
import numpy as np
import PIL

img=tifffile.imread('test.tif')
H,W=img.shape
zmin,zmax=np.min(img),np.max(img)
print(img.shape,img.dtype,zmin,zmax)

# ////////////////////////////////////////////////
def CreateImage(r=None,g=None,b=None):
    if r is None:
        r=(np.random.random([256,256])*255.0).astype(np.uint8)
        g=np.copy(r)
        b=np.copy(r)
    return PIL.Image.fromarray(np.stack([r,g,b],axis=-1))


import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import widgets

show_min        = widgets.BoundedIntText(value=0    , min=zmin, max=zmax, step=1, description=f'Min/{zmin}')
show_max        = widgets.BoundedIntText(value=10000, min=zmin, max=zmax, step=1, description=f'Max/{zmax}')
show_saturation = widgets.Checkbox(description='Show saturation: ',value=True)

fig = go.Figure()
fig.layout.dragmode='pan'

fig.add_trace(go.Scatter(x=[0, W ],y=[0, H],mode="markers",marker_opacity=0)) # This trace is added to help the autoresize logic work.
fig.update_xaxes(visible=False,range=[0, W])
fig.update_yaxes(visible=False,range=[0, H],scaleanchor="x")

# ////////////////////////////////////////////////
def RefreshImage(widget, evt=None):
    print(f"RefreshColors min={show_min.value} max={show_max.value} show_staturation={show_saturation.value} ")
    r=g=b=(255.0*(img/zmax)).astype(np.uint8)
    if show_saturation.value: 
        g,b=np.copy(r),np.copy(r)
        mask=r>(255.0*(show_max.value/zmax))
        r[mask],g[mask],b[mask]=255,0,0
    new_image=CreateImage(r,g,b)
    widget.update_layout_images(source=new_image)

fig.add_layout_image(dict(
    x=0, y=H, sizex=W, sizey=H,
    xref="x",yref="y",opacity=1.0, layer="below",sizing="stretch",
    source=CreateImage()
))
fig.update_layout(margin={"l": 0, "r": 0, "t": 0, "b": 0})

g=go.FigureWidget(fig)
g.set_trait("_config",{"scrollZoom": True})  # if you cannot debug, comment this line
show_min       .observe(lambda change: RefreshImage(g), names="value")
show_max       .observe(lambda change: RefreshImage(g), names="value")
show_saturation.observe(lambda change: RefreshImage(g), names="value")

RefreshImage(g)
widgets.HBox([
    g,
    widgets.VBox([
        show_min,
        show_max,
        show_saturation])
])

(3889, 3073) float32 0.0 16095.0
RefreshColors min=0 max=10000 show_staturation=True 


    'data': [{'marker': {'opacity': 0},
              'mode': 'markers',
       …

RefreshColors min=0 max=10000 show_staturation=False 
RefreshColors min=0 max=10000 show_staturation=True 
RefreshColors min=0 max=16000 show_staturation=True 
RefreshColors min=0 max=16095 show_staturation=True 
RefreshColors min=0 max=16094 show_staturation=True 
RefreshColors min=0 max=16090 show_staturation=True 
RefreshColors min=0 max=16088 show_staturation=True 
RefreshColors min=0 max=15088 show_staturation=True 


In [9]:
f=open("log.out","w")

import time
T1=time.time()

def Write(msg):
    sec=time.time()-T1
    msg=f"[{sec}] {msg}\n"
    f.write(msg)
    f.flush()

In [26]:
import panel as pn
pn.extension()

import time
import tifffile
import numpy as np
import PIL

from bokeh.plotting import figure

#H,W=512,512
#img=np.random.random([H,W])*16049.0
img=tifffile.imread('test.tif')
H,W=img.shape

zmin,zmax=np.min(img),np.max(img)
print(img.shape,img.dtype,zmin,zmax)

show_min        = pn.widgets.IntInput(value=0    , name=f'Min/{zmin}',start=int(zmin),end=int(zmax),step=1)
show_max        = pn.widgets.IntInput(value=10000, name=f'Max/{zmax}',start=int(zmin),end=int(zmax),step=1)
show_saturation = pn.widgets.CheckBoxGroup(value=['Saturation'], options=["Saturation"])

last_renderer=None

T1=time.time()

# ////////////////////////////////////////////////
def CreateImage(r=None,g=None,b=None):
    Write("Creating image..")
    if r is None: 
        r=g=b=(np.random.random([H,W])*255.0).astype(np.uint8)
    h,w = r.shape
    img = np.empty((h,w), dtype=np.uint32)
    view = img.view(dtype=np.uint8).reshape((h, w, 4))
    view[:,:,0:3]=np.stack([r,g,b],axis=-1)
    view[:,:,  3]=255
    Write("DONE")
    return img

# ////////////////////////////////////////////////
def SetImage(rgb):
    Write("SetImage..")
    global last_renderer
    if last_renderer:
        fig.renderers.remove(last_renderer)
    last_renderer=fig.image_rgba(image=[rgb], x=0, y=0, dw=10, dh=10)
    Write("DONE")

# ////////////////////////////////////////////////
def RefreshImage(evt):
    Write("Generating saturation..")
    r=g=b=(255.0*(img/zmax)).astype(np.uint8)
    if show_saturation.value: 
        g,b=np.copy(r),np.copy(r)
        mask=r>(255.0*(int(show_max.value)/zmax))
        r[mask],g[mask],b[mask]=255,0,0
    Write("Done..")
    rgb=CreateImage(r,g,b) 
    SetImage(rgb)

fig = figure(width=600, height=600, active_scroll="wheel_zoom")
fig.x_range.range_padding = fig.y_range.range_padding = 0
SetImage(CreateImage(None))

show_min       .param.watch(RefreshImage, 'value')
show_max       .param.watch(RefreshImage, 'value')
show_saturation.param.watch(RefreshImage, 'value')

pn.Row(
    pn.Row(
        pn.pane.Bokeh(fig)
    ),
    pn.Column(
        show_min,
        show_max,
        show_saturation
    ))

(3889, 3073) float32 0.0 16095.0


BokehModel(combine_events=True, render_bundle={'docs_json': {'b0905d05-c555-4181-9434-77ae73c03e88': {'version…